<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

!{sys.executable} -m pip install --force-reinstall --upgrade "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"
import craid.eddb.loader.DataProducer as dp
import pandas as pd
import logging
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import timedelta, date


logging.getLogger().addHandler(logging.StreamHandler())
logging.getLogger().level = logging.DEBUG

arrays = dp.getDataArrays()
baseFrame: pd.DataFrame = arrays['dataFrame']

  Cloning https://github.com/HausReport/ClubRaiders.git to /tmp/pip-install-ipnhaaqz/craid
  Running command git clone -q https://github.com/HausReport/ClubRaiders.git /tmp/pip-install-ipnhaaqz/craid
     |████████████████████████████████| 71kB 3.1MB/s 
     |████████████████████████████████| 3.4MB 6.3MB/s 
     |████████████████████████████████| 194kB 52.9MB/s 
     |████████████████████████████████| 1.8MB 37.6MB/s 
     |████████████████████████████████| 1.2MB 54.4MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
     |████████████████████████████████| 10.1MB 41.8MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
     |████████████████████████████████| 798kB 59.2MB/s 
     |████████████████████████████████| 14.6MB 52.1MB/s 
     |████████████████████████████████| 235kB 56.0MB/s 
     |████████████████████████████████| 184kB 56.3MB/s 
     |████████████████████████████████| 133kB 55.6MB/s 
     |████████████████████████████████| 450kB 56.6MB/s 
     |██████████████

Loading from AWS
1- downloading to: /tmp/smol-factions.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-factions.jsonl.gz] to [/tmp/smol-factions.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443


0:212,221,952


https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-factions.jsonl.gz HTTP/1.1" 200 15372
Found data file: /tmp/smol-factions.jsonl.gz
Read 659 lines of faction data
1- downloading to: /tmp/smol-systems_populated.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-systems_populated.jsonl.gz] to [/tmp/smol-systems_populated.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443


0.5:213,921,792


https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-systems_populated.jsonl.gz HTTP/1.1" 200 25613
Found data file: /tmp/smol-systems_populated.jsonl.gz
Read 171 lines of systems data
Populated club system keys
Club system lookup set has {171} items
Populated faction instances: {1036} items.
1- downloading to: /tmp/smol-stations.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-stations.jsonl.gz] to [/tmp/smol-stations.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443


1:216,612,864
2:217,690,112


https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-stations.jsonl.gz HTTP/1.1" 200 133539
Found data file: /tmp/smol-stations.jsonl.gz
Read 768 lines of station data, adding 768

station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research Base
station controlling faction doesn't exist: Qwent Research B

3:218,419,200


Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
from craid.eddb.loader.strategy.DataLoader import DataLoader
from craid.eddb.loader.strategy.GithubLoader import LoadDataFromGithub
from craid.history.History import History

myLoader = LoadDataFromGithub(_forceWebDownload=True, useSmol=False)
hist = History()
csa = hist.getRawDataFrame()
target = hist.getNormalizedDataFrame()

1- downloading to: /tmp/history.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/history.jsonl.gz] to [/tmp/history.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443


Creating the object


https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /history.jsonl.gz HTTP/1.1" 200 85315
Found data file: /tmp/history.jsonl.gz
Read system        7591
faction       7591
updated       7591
influence     7591
control       7591
region        7591
population    7591
dtype: int64 lines of history data


In [3]:
# data cleaning
# done in api now
#csa = csa.drop_duplicates()
#csa = csa[ csa['faction'] != 'Aegis of Federal Democrats']
#csa = csa[ csa['faction'] != 'Aegis Imperium']
#csa = csa[ csa['faction'] != "Emperor's Dawn"]

In [4]:
target[ target['updated'] == "2020-06-12"]

,system,faction,updated,influence,control,region,population
3560,Abroin,Abroin Universal PLC,2020-06-12,46.8594,True,6,2813462
4736,HR 1183,Aegis Research,2020-06-12,0.9775,False,1,1200000
4774,HR 3714,Sirius Luxury Transports,2020-06-12,4.5000,False,5,14701561
5389,Mel 22 Sector GM-V c2-8,Turner Research Group,2020-06-12,100.0000,True,0,18550
3560,Abroin,Abroin Universal PLC,2020-06-12,46.8594,True,6,2813462
4736,HR 1183,Aegis Research,2020-06-12,0.9775,False,1,1200000
4774,HR 3714,Sirius Luxury Transports,2020-06-12,4.5000,False,5,14701561
5389,Mel 22 Sector GM-V c2-8,Turner Research Group,2020-06-12,100.0000,True,0,18550
3560,Abroin,Abroin Universal PLC,2020-06-12,46.8594,True,6,2813462
4736,HR 1183,Aegis Research,2020-06-12,0.9775,False,1,1200000


In [5]:
stage1 = csa.loc[csa.groupby(['faction','system']).updated.idxmax()]
stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
fig = px.pie(stage3, values='influence', names='faction', title='Total Influence by Minor Faction', width=350, height=350)
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [6]:
from craid.club.regions.RegionFactory import RegionFactory
stage3 = stage1[['region','influence']].groupby('region').influence.sum().reset_index()
regionNames = list(RegionFactory.regionDict.keys())
fig = px.pie(stage3, values='influence', names=regionNames, title='Total Club Influence by Region', width=350, height=350) #, names='faction'
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [7]:
region = 1
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence <br>by Minor Faction <br>in the {regionName} region"
merope = target[ target['region']== region]
stage1 = merope.loc[merope.groupby(['faction','system']).updated.idxmax()]
stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
fig = px.pie(stage3, values='influence', names='faction', title=title, width=350, height=350) #, names='faction'
fig.update_traces(textposition="inside")
fig.update_layout(showlegend=False)
fig.show()

In [10]:
theDate = '2020-06-1'
#reg = csa[ csa['region']== 1]
dated = csa [ csa['updated']<= theDate]

stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
#stage1[['system','faction','influence']]
stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
stage3.influence.sum()

4168.610099040681

In [11]:
region = 8

dates = []
inf = []

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) 

dframe = csa[ csa['region'] == region]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Club Influence in the {regionName} region"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)
      
labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [12]:

dates = []
inf = []

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) 

dframe = csa
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Club Influence"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)

labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [13]:
dates = []
inf = []

faction = "Sirius Corporation"
faction = "Hodack Prison Colony"
faction = "Wreaken Construction"
#faction = "'Sirius Drives'"
faction = "Aegis Core"
#faction = "Aegis Research"
#faction = "Bill Turner"

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) #date(2020, 6, 28)


dframe = csa[ csa['faction'] == faction]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence of the {faction} Faction"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)


labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [14]:
badGuys = ['Abroin Universal PLC', 'Aegis Core', 'Aegis Research',
       "Benton's Gang", 'Bentonian Party', 'CQC Holdings',
       'Gallant Investment Brokers', 'Hodack Prison Colony',
       'Janus Incorporated', "Namarii Emperor's Dawn", 'Reyan BPS',
       'Reynhardt IntelliSys', 'Sirius Atmospherics', 'Sirius Catering',
       'Sirius Corporation', 'Sirius Drives', 'Sirius Hot2Cold',
       'Sirius Hyperspace', 'Sirius Industrial',
       'Sirius Luxury Transports', 'Sirius Mining Merope',
       'Sirius Mining', 'Sirius Power', 'The Greenventure Group',
       'The Peterson Group', 'The Rockforth Corporation',
       'Turner Research Group', 'Wiggins Development Trust',
       'Worster Insurance', 'Wreaken Construction', 'Bill Turner',
       'Aegis Defense']

In [15]:
a = csa['faction'].unique().tolist()
b = baseFrame['factionName'].unique().tolist()
aa = set(a)
bb = set(b)
bb-aa

set()

In [16]:
dates = []
inf = []

faction = "Combined Wreaken and Hodack"
faction1 = "Wreaken Construction"
faction2 = "Hodack Prison Colony"

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) #date(2020, 6, 28)


dframe = csa[ (csa['faction'] == faction1) | (csa['faction'] == faction2) ]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence of the {faction} Faction"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)


labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [17]:
dates = []
inf = []

faction = "Combined Sirius Factions"
faction1 = "Sirius"

start_date = date(2018, 5, 15)
daily_date = date(2020,5,30)
end_date = date.today() + timedelta(days=1) #date(2020, 6, 28)


dframe = csa[ (csa['faction'].str.contains(faction1)) ]
regionNames = list(RegionFactory.regionDict.keys())
regionName = regionNames[region]
title = f"Total Influence of the {faction} Faction"

single_date = start_date
while single_date <= end_date:
    theDate = single_date.strftime("%Y-%m-%d")
    dated = dframe [ dframe['updated']<= theDate]

    stage1 = dated.loc[dated.groupby(['faction','system']).updated.idxmax()]
    stage3 = stage1[['system','faction','influence']].groupby('faction').influence.sum().reset_index()
    theInf = stage3.influence.sum()
    dates.append(theDate)
    inf.append(theInf)
    if(single_date<daily_date):
      single_date = single_date + relativedelta(months=1)
    else:
      single_date = single_date + relativedelta(days=1)


labels = {'x':'Day', 'y':"Total Influence"}
fig = px.line(x=dates, y=inf, title=title, labels=labels)
fig.show()

In [18]:
def getSystemInfluenceGraph(sys: str):
  ogun = target[ target['system']==sys]
  a = ogun['faction'].unique().tolist()
  fig = go.Figure()
  for fac in a:
    theSlice = ogun[ ogun['faction'] == fac].reset_index()
    fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    name=fac))
  return fig

In [19]:
# One System, handles Multiple Factions
fig = getSystemInfluenceGraph("Celaeno")
fig.show()

In [20]:
fig = getSystemInfluenceGraph("Solati")
fig.show()

In [21]:
# One System, handles Multiple Factions
fig = getSystemInfluenceGraph("Sol")
fig.show()

In [22]:
fac = "Hodack Prison Colony"
ogun = target[ target['faction']==fac]
a = ogun['system'].unique().tolist()
fig = go.Figure()
for sys in a:
  theSlice = ogun[ ogun['system'] == sys].reset_index()
  fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    name=sys,
                    stackgroup="one"))
  # Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)
fig.show()


In [23]:
def getFactionInfluenceGraph(fac: str):
  ogun = target[ target['faction']==fac]
  a = ogun['system'].unique().tolist()
  fig = go.Figure()
  for sys in a:
    theSlice = ogun[ ogun['system'] == sys].reset_index()
    fig.add_trace(go.Scatter(x=theSlice['updated'], y=theSlice['influence'],
                    mode='lines',
                    stackgroup="b",
                    name=sys))
  fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
  )
  return fig

In [24]:
getFactionInfluenceGraph('Hodack Prison Colony').show()

In [29]:
fac = 'Sirius Mining'
fac = 'Sirius Mining Merope'
fac = 'CQC Holdings'
getFactionInfluenceGraph(fac).show()

In [28]:
target.drop(target[target.updated == "2020-06-11"].index, inplace=True)
target.drop(target[target.updated == "2020-06-27"].index, inplace=True)

